In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from patterns import * 
import talib

In [3]:
# Load the data into a Pandas DataFrame
df = pd.read_csv(
    Path('datasets/BTCUSD_1h.csv'),
    index_col="Date")
df.tail()

,Open,Close,High,Low,Volume
Date,,,,,
2021-11-17 13:00:00,60514.231682,60207.000000,60639.0,60207.0,476.771475
2021-11-17 14:00:00,60207.000000,59724.666301,60388.0,59724.0,824.163709
2021-11-17 15:00:00,59724.000000,59708.630836,59859.0,59483.0,500.389504
2021-11-17 16:00:00,59728.000000,60418.000000,60480.0,59491.0,398.163219
2021-11-17 17:00:00,60418.000000,60069.000000,60439.0,60031.0,109.235020


In [4]:
net_liquid_value = 100000
max_loss_single = 0.005
max_loss_total = max_loss_single * 10

initial_win_percentage = 0.5
initial_loss_percentage = 1 - initial_win_percentage

average_winner = 100
average_loser = 75

trade_size_percentage = ((initial_win_percentage * average_winner) - (initial_loss_percentage * average_loser)) / 100
trade_size = net_liquid_value * trade_size_percentage
trade_size

12500.0

In [5]:
# extract OHLC 
op = df['Open'].astype(float)
hi = df['High'].astype(float)
lo = df['Low'].astype(float)
cl = df['Close'].astype(float) 


candle_names = talib.get_function_groups()['Pattern Recognition']
engulfing_candle = talib.CDLENGULFING(op, hi, lo, cl)
# create columns for each candle
for candle in engulfing_candle:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df[candle] = engulfing_candle #getattr(talib, candle)(op, hi, lo, cl)
    
df.tail()

,Open,Close,High,Low,Volume,0,100,-100
Date,,,,,,,,
2021-11-17 13:00:00,60514.231682,60207.000000,60639.0,60207.0,476.771475,0,0,0
2021-11-17 14:00:00,60207.000000,59724.666301,60388.0,59724.0,824.163709,0,0,0
2021-11-17 15:00:00,59724.000000,59708.630836,59859.0,59483.0,500.389504,0,0,0
2021-11-17 16:00:00,59728.000000,60418.000000,60480.0,59491.0,398.163219,0,0,0
2021-11-17 17:00:00,60418.000000,60069.000000,60439.0,60031.0,109.235020,0,0,0


In [6]:
df['entry'] = np.where(df[0]>0,1,0)
df = df[['Open', 'Close', 'High', 'Low', 'Volume', 'entry']]
df.head()

,Open,Close,High,Low,Volume,entry
Date,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,1
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0


In [7]:
df['exit'] = df['entry'].shift(3)



In [8]:
df['exit'].sum()

1806.0

In [9]:
df.head(10)

,Open,Close,High,Low,Volume,entry,exit
Date,,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0,NaN
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,NaN
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,1,NaN
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0.0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0.0
2017-01-01 12:00:00,977.11,973.32,978.59,970.85,292.530740,0,1.0
2017-01-01 13:00:00,973.98,976.89,978.51,973.75,82.366898,0,0.0
2017-01-01 14:00:00,976.58,978.45,978.49,976.00,115.830580,0,0.0
2017-01-01 15:00:00,978.50,981.96,982.00,976.46,459.671480,0,0.0


In [10]:
df['entry_exit'] = np.nan

for index, row in df.iterrows():
    if row['entry'] == 1:
        df.loc[index, 'entry_exit'] = 1
    elif row['exit'] == 1:
        df.loc[index, 'entry_exit'] = -1
    else:
        df.loc[index, 'entry_exit'] = 0

In [11]:
df.head(10)

,Open,Close,High,Low,Volume,entry,exit,entry_exit
Date,,,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0,NaN,0.0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,NaN,0.0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,1,NaN,1.0
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0.0,0.0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0.0,0.0
2017-01-01 12:00:00,977.11,973.32,978.59,970.85,292.530740,0,1.0,-1.0
2017-01-01 13:00:00,973.98,976.89,978.51,973.75,82.366898,0,0.0,0.0
2017-01-01 14:00:00,976.58,978.45,978.49,976.00,115.830580,0,0.0,0.0
2017-01-01 15:00:00,978.50,981.96,982.00,976.46,459.671480,0,0.0,0.0


In [12]:
net_liquid_value = 100000
max_loss_single = 0.005
max_loss_total = max_loss_single * 10

initial_win_percentage = 0.5
initial_loss_percentage = 1 - initial_win_percentage

average_winner = 100
average_loser = 75

trade_size_percentage = ((initial_win_percentage * average_winner) - (initial_loss_percentage * average_loser)) / 100
trade_size = net_liquid_value * trade_size_percentage
trade_size

12500.0

In [21]:
df['holding'] = np.nan
df['shares'] = np.nan
# df['win'] = np.nan
# df['loss'] = np.nan
# df['P/L'] = np.nan
# df['win_dollars'] = np.nan
# df['lose_dollars'] = np.nan
# df['win_pct'] = np.nan
# df['lose_pct'] = np.nan

net_liquid_value = 100000
max_loss_single = 0.005
max_loss_total = max_loss_single * 10

initial_win_percentage = 0.5
initial_loss_percentage = 1 - initial_win_percentage


number_of_shares = 0
previous_cost = 0
portfolio_value = net_liquid_value
win = 1
loss = 1
win_pct = win / (win + loss)
lose_pct = 1 - win_pct

winners = 0
losers = 0

average_loser = losers / loss
average_winner = winners / win

trade_size_percentage = ((win_pct * average_winner) - (lose_pct * average_loser)) / 100
trade_size = portfolio_value * trade_size_percentage

size = 12500
for index, row in df.iterrows():
    if row['entry_exit'] == 1:
        share_size = round(size / row['Open'], 0)
        df.loc[index, 'shares'] = share_size
        number_of_shares += share_size
        previous_cost = share_size * row['Open']
        win += 0
        loss += 0
        winners += 0
        losers += 0 
        df.loc[index, 'win'] = win
        df.loc[index, 'loss'] = loss
        # df.loc[index, 'shares'] = share_size
        df.loc[index, 'holding'] = portfolio_value
        # df.loc[index, 'P/L'] = 0
        # df.loc[index, 'lose_dollars'] = 0
        # df.loc[index, 'win_dollars'] = 0
        # df.loc[index, 'win_pct'] = win_pct
        # df.loc[index, 'loss_pct'] = lose_pct
    elif row['entry_exit'] == -1:
        current_cost = (row['Close'] * number_of_shares) - previous_cost
        if current_cost > 0:
            win += 1
            loss += 0
            winners += current_cost
            losers += 0
            df.loc[index, 'win'] = win
            df.loc[index, 'loss'] = loss
            # df.loc[index, 'win_pct'] = win_pct
            # df.loc[index, 'loss_pct'] = lose_pct
        elif current_cost < 0:
            win += 0
            loss += 1
            winners += 0
            losers += current_cost
            df.loc[index, 'loss'] = loss
            df.loc[index, 'win'] = win
            # df.loc[index, 'win_pct'] = win_pct
            # df.loc[index, 'loss_pct'] = lose_pct
        else:
            win += 0
            loss += 0
            winners += 0
            losers += 0            
            df.loc[index, 'loss'] = 0
            df.loc[index, 'win'] = 0
            # df.loc[index, 'win_pct'] = win_pct
            # df.loc[index, 'loss_pct'] = lose_pct
        df.loc[index, 'shares'] = 0
        number_of_shares = 0        
        portfolio_value += current_cost
        df.loc[index, 'holding'] = portfolio_value
        previous_cost = 0
    else:
        win += 0
        loss += 0
        winners += 0
        losers += 0 
        df.loc[index, 'shares'] = number_of_shares
        df.loc[index, 'holding'] = portfolio_value


        
# for index, row in df.iterrows():
#     if row['P/L'] > 0:
#         df.loc[index, 'win_dollars'] = row['P/L']
#         df.loc[index, 'lose_dollars'] = 0
#     elif row['P/L'] < 0:
#         df.loc[index, 'lose_dollars'] = row['P/L']
#         df.loc[index, 'win_dollars'] = 0
#     else:
#         df.loc[index, 'lose_dollars'] = row['P/L']
#         df.loc[index, 'win_dollars'] = row['P/L']



# for index, row in df.iterrows():
#     df.loc[index, 'win_pct'] = row['win'] / (row['win'] + row['loss'])
#     df.loc[index, 'lose_pct'] = row['loss'] / (row['win'] + row['loss'])



In [36]:
# number_of_shares
# previous_cost 
# portfolio_value 
# win
# # loss
win_pct
# # lose_pct

# winners
# losers

# average_loser 
# average_winner 

# trade_size_percentage 
# trade_size 

0.5

In [23]:
df.head(50)

,Open,Close,High,Low,Volume,entry,exit,entry_exit,holding,shares,win,loss
Date,,,,,,,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0,NaN,0.0,100000.00,0.0,NaN,NaN
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,NaN,0.0,100000.00,0.0,NaN,NaN
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,1,NaN,1.0,100000.00,13.0,1.0,1.0
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0.0,0.0,100000.00,13.0,NaN,NaN
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0.0,0.0,100000.00,13.0,NaN,NaN
2017-01-01 12:00:00,977.11,973.32,978.59,970.85,292.530740,0,1.0,-1.0,100102.83,0.0,2.0,1.0
2017-01-01 13:00:00,973.98,976.89,978.51,973.75,82.366898,0,0.0,0.0,100102.83,0.0,NaN,NaN
2017-01-01 14:00:00,976.58,978.45,978.49,976.00,115.830580,0,0.0,0.0,100102.83,0.0,NaN,NaN
2017-01-01 15:00:00,978.50,981.96,982.00,976.46,459.671480,0,0.0,0.0,100102.83,0.0,NaN,NaN


In [15]:
#df['win'].sum()
df['loss'].sum()

KeyError: 'loss'

In [ ]:
df.tail(20)